In [0]:
import os, re, requests
from datetime import datetime


# Parâmetro do Job
dbutils.widgets.text("DATA_ALVO", "")
DATA_ALVO = dbutils.widgets.get("DATA_ALVO") or datetime.now().strftime("%Y%m%d")


BASE_URL = "https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/10min/"
DESTINO = f"/Volumes/datamasters/raw/raw_inpe/inpe_in/{DATA_ALVO}"
HEADERS = {"User-Agent": "inpe-batch/1.0"}


os.makedirs(DESTINO, exist_ok=True)


html = requests.get(BASE_URL, headers=HEADERS).text
csvs = re.findall(r'"([^"]+\.csv)"', html)


for nome in csvs:
if DATA_ALVO in nome:
url = BASE_URL + nome
content = requests.get(url, headers=HEADERS).content
path = f"/tmp/{nome}"
with open(path, "wb") as f:
f.write(content)
dbutils.fs.cp(f"file:{path}", f"dbfs:{DESTINO}/{nome}")